In [51]:
from keras.applications import inception_v3
from keras import backend as K

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [72]:
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                               include_top=False)

In [73]:
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}
    

In [74]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)

for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + (coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling)

In [95]:
dream = model.input

grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Wartość straty, i, ":"', loss_value)
        x += step*grad_values
    return x
        

In [96]:
import imageio
import scipy
import scipy.misc
from keras_preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)
    
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() =='channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x += 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [97]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.

base_image_path = 'nowa_luda.png'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Zmiana kształtu obrazu', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    
    same_size_original = resize_img(original_img, shape)
    
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='final_dream.png')


Zmiana kształtu obrazu (93, 148)
...Wartość straty, i, ":" 0.050811592
...Wartość straty, i, ":" 0.05845772
...Wartość straty, i, ":" 0.07096273
...Wartość straty, i, ":" 0.082252406
...Wartość straty, i, ":" 0.09494557
...Wartość straty, i, ":" 0.10628264
...Wartość straty, i, ":" 0.11711091
...Wartość straty, i, ":" 0.12692024
...Wartość straty, i, ":" 0.13819952
...Wartość straty, i, ":" 0.14759107
...Wartość straty, i, ":" 0.15707707
...Wartość straty, i, ":" 0.16530702
...Wartość straty, i, ":" 0.175092
...Wartość straty, i, ":" 0.1814034
...Wartość straty, i, ":" 0.18976784
...Wartość straty, i, ":" 0.19651154
...Wartość straty, i, ":" 0.20526695
...Wartość straty, i, ":" 0.21310867
...Wartość straty, i, ":" 0.21765798
...Wartość straty, i, ":" 0.22688058
Zmiana kształtu obrazu (131, 208)
...Wartość straty, i, ":" 1.2146258
...Wartość straty, i, ":" 1.7317538
...Wartość straty, i, ":" 2.2204368
...Wartość straty, i, ":" 2.6468954
...Wartość straty, i, ":" 3.0888855
...Wartość str